<a href="https://colab.research.google.com/github/AbR04/5A-AI/blob/main/WEEK_9_AI_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from typing import List, Tuple

def is_complementary(clause1: set, clause2: set) -> Tuple[bool, set]:
    """
    Check if two clauses are complementary and return the resolved clause if they are.
    """
    for literal in clause1:
        if f"~{literal}" in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(f"~{literal}")
            return True, resolved
        elif literal.startswith("~") and literal[1:] in clause2:
            resolved = clause1.union(clause2)
            resolved.discard(literal)
            resolved.discard(literal[1:])
            return True, resolved
    return False, set()

def resolution(kb: List[set], goal: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    """
    Perform resolution and build the proof tree.
    """
    proof_tree = []
    clauses = kb + [goal]  # Add negated goal to the KB
    new_clauses = []

    while True:
        # Generate new pairs of clauses
        pairs = [(clauses[i], clauses[j], (i, j)) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]

        for clause1, clause2, (i, j) in pairs:
            complementary, resolved_clause = is_complementary(clause1, clause2)
            if complementary:
                if not resolved_clause:  # Empty clause found
                    proof_tree.append((resolved_clause, [i, j]))
                    return True, proof_tree
                if resolved_clause not in clauses and resolved_clause not in new_clauses:
                    new_clauses.append(resolved_clause)
                    proof_tree.append((resolved_clause, [i, j]))

        if not new_clauses:  # No progress, resolution failed
            return False, proof_tree

        clauses.extend(new_clauses)
        new_clauses = []

# Example case study
if __name__ == "__main__":
    # Knowledge Base (KB) in CNF format
    kb = [
        {"P", "Q"},       # Clause 1: P ∨ Q
        {"~P", "R"},      # Clause 2: ¬P ∨ R
        {"~R"},           # Clause 3: ¬R
    ]
    goal = {"~Q"}  # Negated goal: ¬Q

    success, proof_tree = resolution(kb, goal)
    if success:
        print("PROVED")
    else:
        print("CAN'T BE PROVED")

    print("\nProof Tree:")
    for i, (clause, parents) in enumerate(proof_tree):
        print(f"Step {i + 1}: {clause} from {parents}")

PROVED

Proof Tree:
Step 1: {'Q', 'R'} from [0, 1]
Step 2: {'P'} from [0, 3]
Step 3: {'~P'} from [1, 2]
Step 4: {'Q'} from [0, 6]
Step 5: {'R'} from [1, 5]
Step 6: set() from [5, 6]
Game Tree Before Alpha-Beta Pruning:
Node Value: None
  Node Value: None
    Node Value: None
      Node Value: 10
      Node Value: 9
    Node Value: None
      Node Value: 14
      Node Value: 18
  Node Value: None
    Node Value: None
      Node Value: 5
      Node Value: 4
    Node Value: None
      Node Value: 50
      Node Value: 3
Pruned at MAX node with alpha=14, beta=10
Pruned at MIN node with alpha=10, beta=5

Game Tree After Alpha-Beta Pruning:
Node Value: 10
  Node Value: 10
    Node Value: 10
      Node Value: 10
      Node Value: 9
    Node Value: 14
      Node Value: 14
      Node Value: 18
  Node Value: 5
    Node Value: 5
      Node Value: 5
      Node Value: 4
    Node Value: None
      Node Value: 50
      Node Value: 3

Final Value at MAX node: 10


In [4]:
from itertools import combinations

# Helper function to unify two terms (simple version)
def unify(term1, term2):
    if term1 == term2:
        return {}
    if isinstance(term1, str) and term1.islower():  # variable case
        return {term1: term2}
    if isinstance(term2, str) and term2.islower():  # variable case
        return {term2: term1}
    return None  # Cannot unify

# Function to resolve two clauses with predicates
def pl_resolve(clause1, clause2):
    resolvents = set()

    # Try to unify complementary literals between the two clauses
    for literal1 in clause1:
        for literal2 in clause2:
            # Check for complementary literals: `A(x)` and `¬A(x)`
            if literal1 == f"¬{literal2}":
                # Attempt to unify the literals
                unification = unify(literal1[1:], literal2)
                if unification is not None:
                    # Apply the unification to both clauses
                    resolved_clause = (clause1 - {literal1}) | (clause2 - {literal2})
                    # Substitute the unification in the resolved clause
                    resolved_clause = {apply_substitution(literal, unification) for literal in resolved_clause}
                    resolvents.add(frozenset(resolved_clause))
    return resolvents

# Function to apply a unification to a term
def apply_substitution(term, substitution):
    if term in substitution:
        return substitution[term]
    return term

# Convert first-order logic statements to clauses in CNF
def to_clauses(kb):
    clauses = []
    for expr in kb:
        clauses.append(frozenset(expr))
    return set(clauses)  # Ensure it's a set for union operations

# Function to negate the query
def negate_query(query):
    # Negating a predicate like `older(leela, Oslin)` to `¬older(leela, Oslin)`
    return f"¬{query}"

# The PL-Resolution function (with unification and handling predicates)
def pl_resolution(KB, query):
    # Step 1: Negate the query (to prove negation by resolution)
    neg_query = negate_query(query)

    # Step 2: Convert the KB and the negated query into a set of clauses (CNF)
    clauses = to_clauses(KB) | {frozenset([neg_query])}

    new = set()  # Set to store new clauses
    while True:
        # Step 3: Resolve all pairs of clauses in the knowledge base
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = pl_resolve(clause1, clause2)
            if frozenset():  # If the empty clause is found, return true (contradiction)
                return True
            new |= resolvents

        # Step 4: Check if new clauses are already in the knowledge base (no progress)
        if new.issubset(clauses):  # No new clauses, termination
            return False

        # Step 5: Add the new clauses to the knowledge base
        clauses |= new

# Example Usage

def main():
    # Define the knowledge base
    KB = [
        {"Mother(leela, Oslin)"},   # Mother(leela, Oslin)
        {"Alive(leela)"},           # Alive(leela)
        {"Mother(x, y) → Parent(x, y)"},  # ∀x∀y Mother(x, y) → Parent(x, y)
        {"Parent(x, y)", "Alive(x)", "→", "Older(x, y)"}  # ∀x∀y (Parent(x, y) and Alive(x)) → Older(x, y)
    ]

    # Define the query to prove: older(leela, Oslin)
    query = "Older(leela, Oslin)"

    # Apply PL-Resolution to prove the query
    result = pl_resolution(KB, query)
    print("Query is satisfiable:", result)

if __name__ == "__main__":
    main()


Query is satisfiable: False
